In [2]:
import pandas as pd
import sys  
import os
import numpy as np

In [4]:
df_credit = pd.read_csv('credit_card_balance_KPI_final.csv')
df_installments = pd.read_csv('installment_kpi_final.csv')

In [ ]:
df_credit

In [10]:


df = df_credit.copy()

# 1) CC_90PLUS : 90일 이상 연체 횟수
cc_90plus = df.groupby('SK_ID_CURR')['SK_DPD'].apply(lambda x: (x >= 90).sum()).reset_index()
cc_90plus.columns = ['SK_ID_CURR', 'CC_90PLUS']

# 2) AVG_SK_DPD : 평균 연체일수
avg_sk_dpd = df.groupby('SK_ID_CURR')['SK_DPD'].mean().reset_index()
avg_sk_dpd.columns = ['SK_ID_CURR', 'AVG_SK_DPD']

# 3) AVG_CARD_USAGE_MONTHS : 카드 사용 개월 수 (SK_ID_PREV별 개월수 평균)
card_months = df.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['MONTHS_BALANCE'].count().reset_index()
avg_card_usage_months = card_months.groupby('SK_ID_CURR')['MONTHS_BALANCE'].mean().reset_index()
avg_card_usage_months.columns = ['SK_ID_CURR', 'AVG_CARD_USAGE_MONTHS']

# 4) AVG_UTIL_RATIO : 한도 대비 사용률 평균
df['UTIL_RATIO'] = df['AMT_BALANCE'] / df['AMT_CREDIT_LIMIT_ACTUAL'].replace(0, np.nan)
avg_util_ratio = df.groupby('SK_ID_CURR')['UTIL_RATIO'].mean().reset_index()
avg_util_ratio.columns = ['SK_ID_CURR', 'AVG_UTIL_RATIO']

# 5) AVG_MIN_PAY_RATIO : 최소납부비율 평균
df['MIN_PAY_RATIO'] = df['AMT_INST_MIN_REGULARITY'] / df['AMT_TOTAL_RECEIVABLE'].replace(0, np.nan)
df['MIN_PAY_RATIO'] = df['MIN_PAY_RATIO'].fillna(0)  # 결측 0 채움
avg_min_pay_ratio = df.groupby('SK_ID_CURR')['MIN_PAY_RATIO'].mean().reset_index()
avg_min_pay_ratio.columns = ['SK_ID_CURR', 'AVG_MIN_PAY_RATIO']

# 6) AVG_REPAY_RATIO : 상환비율 평균
df['REPAY_RATIO'] = df['AMT_PAYMENT_TOTAL_CURRENT'] / df['AMT_TOTAL_RECEIVABLE'].replace(0, np.nan)
df['REPAY_RATIO'] = df['REPAY_RATIO'].fillna(0)
avg_repay_ratio = df.groupby('SK_ID_CURR')['REPAY_RATIO'].mean().reset_index()
avg_repay_ratio.columns = ['SK_ID_CURR', 'AVG_REPAY_RATIO']

# 7) CREDIT_CARD_HISTORY_YEARS : 카드 사용 이력 기간 (년)
history = df.groupby('SK_ID_CURR')['MONTHS_BALANCE'].agg(['min', 'max']).reset_index()
history['CREDIT_CARD_HISTORY_YEARS'] = (history['max'] - history['min']) / 12
history = history[['SK_ID_CURR', 'CREDIT_CARD_HISTORY_YEARS']]

# 8) CARD_USAGE_BIN_5P : 고객별 기존 변수는 원본에 있으니 거래별로 가장 많이 나온 값으로 집계 (최빈값)
card_usage_bin_5p = df.groupby('SK_ID_CURR')['CARD_USAGE_BIN_5P'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan).reset_index()

# 병합
from functools import reduce

dfs = [
    cc_90plus,
    avg_sk_dpd,
    avg_card_usage_months,
    avg_util_ratio,
    avg_min_pay_ratio,
    avg_repay_ratio,
    history,
    card_usage_bin_5p
]

credit_card_kpi = reduce(lambda left, right: pd.merge(left, right, on='SK_ID_CURR', how='outer'), dfs)

# 결과 확인
print(credit_card_kpi.head())



   SK_ID_CURR  CC_90PLUS  AVG_SK_DPD  AVG_CARD_USAGE_MONTHS  AVG_UTIL_RATIO  \
0      100006          0    0.000000                    6.0        0.000000   
1      100011          0    0.000000                   74.0        0.302678   
2      100013          0    0.010417                   96.0        0.115301   
3      100021          0    0.000000                   17.0        0.000000   
4      100023          0    0.000000                    8.0        0.000000   

   AVG_MIN_PAY_RATIO  AVG_REPAY_RATIO  CREDIT_CARD_HISTORY_YEARS  \
0           0.000000         0.000000                   0.416667   
1          -0.200854        -1.296625                   6.083333   
2           0.011706         0.039772                   7.916667   
3           0.000000         0.000000                   1.333333   
4           0.000000         0.000000                   0.583333   

  CARD_USAGE_BIN_5P  
0             5~10%  
1            70~75%  
2            90~95%  
3            40~45%  
4     

In [6]:
credit_card_kpi['AVG_MIN_PAY_RATIO'] = credit_card_kpi['AVG_MIN_PAY_RATIO'].fillna(0)

In [11]:
credit_card_kpi.isna().mean().sort_values(ascending=False)


AVG_UTIL_RATIO               0.010748
SK_ID_CURR                   0.000000
CC_90PLUS                    0.000000
AVG_SK_DPD                   0.000000
AVG_CARD_USAGE_MONTHS        0.000000
AVG_MIN_PAY_RATIO            0.000000
AVG_REPAY_RATIO              0.000000
CREDIT_CARD_HISTORY_YEARS    0.000000
CARD_USAGE_BIN_5P            0.000000
dtype: float64

In [8]:
credit_card_kpi['AVG_REPAY_RATIO'] = credit_card_kpi['AVG_REPAY_RATIO'].fillna(0)


In [12]:
credit_card_kpi['AVG_UTIL_RATIO'] = credit_card_kpi['AVG_UTIL_RATIO'].fillna(0)

In [13]:
credit_card_kpi.to_csv('credit_card_kpi_final.csv', index=False)

In [14]:
import pandas as pd

df = df_installments.copy()

# 고객 단위로 집계
installments_kpi = df.groupby('SK_ID_CURR').agg({
    'INST_90PLUS': 'sum',             # 90일 이상 연체 건수 합산
    'avg_payment_delay': 'mean',      # 평균 결제 지연 일수 평균
    'pct_underpaid': 'mean',          # 미납 비율 평균
    'pct_full_payments': 'mean'       # 완납 비율 평균
}).reset_index()


In [ ]:
installments_kpi

In [15]:
installments_kpi['avg_payment_delay'] = installments_kpi['avg_payment_delay'].fillna(0)

In [16]:
installments_kpi.to_csv('installment_kpi_final.csv', index=False)

In [20]:
df_app = pd.read_csv('application_train_EXT_SOURCE.csv')

In [23]:
df_app = df_app.merge(credit_card_kpi, on='SK_ID_CURR', how='inner')

In [24]:
df_app

,SK_ID_CURR,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,CC_90PLUS_x,AVG_SK_DPD_x,AVG_CARD_USAGE_MONTHS_x,AVG_UTIL_RATIO_x,AVG_MIN_PAY_RATIO_x,AVG_REPAY_RATIO_x,CREDIT_CARD_HISTORY_YEARS_x,CARD_USAGE_BIN_5P_x,CC_90PLUS_y,AVG_SK_DPD_y,AVG_CARD_USAGE_MONTHS_y,AVG_UTIL_RATIO_y,AVG_MIN_PAY_RATIO_y,AVG_REPAY_RATIO_y,CREDIT_CARD_HISTORY_YEARS_y,CARD_USAGE_BIN_5P_y
0,100006,NaN,0.650442,NaN,0.0,0.000000,6.0,0.000000,0.000000,0.000000,0.416667,5~10%,0,0.000000,6.0,0.000000,0.000000,0.000000,0.416667,5~10%
1,100011,0.587334,0.205747,0.751724,0.0,0.000000,74.0,0.302678,-0.200854,-1.296625,6.083333,70~75%,0,0.000000,74.0,0.302678,-0.200854,-1.296625,6.083333,70~75%
2,100021,NaN,0.683513,NaN,0.0,0.000000,17.0,0.000000,0.000000,0.000000,1.333333,40~45%,0,0.000000,17.0,0.000000,0.000000,0.000000,1.333333,40~45%
3,100023,NaN,0.586617,0.477649,0.0,0.000000,8.0,0.000000,0.000000,0.000000,0.583333,10~15%,0,0.000000,8.0,0.000000,0.000000,0.000000,0.583333,10~15%
4,100036,0.274422,0.627300,NaN,0.0,0.000000,12.0,0.000000,0.000000,0.000000,0.916667,25~30%,0,0.000000,12.0,0.000000,0.000000,0.000000,0.916667,25~30%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86900,456242,NaN,0.746259,0.406617,0.0,0.000000,8.0,0.691240,0.041030,0.175212,0.583333,10~15%,0,0.000000,8.0,0.691240,0.041030,0.175212,0.583333,10~15%
86901,456244,0.896042,0.789389,0.337673,0.0,0.000000,41.0,0.444874,0.018524,0.407869,3.333333,65~70%,0,0.000000,41.0,0.444874,0.018524,0.407869,3.333333,65~70%
86902,456246,NaN,0.313334,0.735221,0.0,0.000000,8.0,0.097309,0.472287,3.792511,0.583333,10~15%,0,0.000000,8.0,0.097309,0.472287,3.792511,0.583333,10~15%
86903,456247,0.243466,0.501221,0.609276,0.0,0.031579,95.0,0.161225,1.013654,1.012195,7.833333,85~90%,0,0.031579,95.0,0.161225,1.013654,1.012195,7.833333,85~90%


In [25]:
df_app.value_counts

<bound method DataFrame.value_counts of        SK_ID_CURR  EXT_SOURCE_1  EXT_SOURCE_2  EXT_SOURCE_3  CC_90PLUS_x  \
0          100006           NaN      0.650442           NaN          0.0   
1          100011      0.587334      0.205747      0.751724          0.0   
2          100021           NaN      0.683513           NaN          0.0   
3          100023           NaN      0.586617      0.477649          0.0   
4          100036      0.274422      0.627300           NaN          0.0   
...           ...           ...           ...           ...          ...   
86900      456242           NaN      0.746259      0.406617          0.0   
86901      456244      0.896042      0.789389      0.337673          0.0   
86902      456246           NaN      0.313334      0.735221          0.0   
86903      456247      0.243466      0.501221      0.609276          0.0   
86904      456248           NaN      0.530725           NaN          0.0   

       AVG_SK_DPD_x  AVG_CARD_USAGE_MONTHS_x  A